In [5]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 63.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f948084ca6ffc1ac962af60a707d5f91c0e855e58e54aabcecc7ed034e07794c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.version

'3.3.1'

In [7]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS


In [13]:
# Load the data into RDD
data = sc.textFile('/content/drive/MyDrive/kuliah/Smester 5/big data lanjut/p11/ml_ratings.txt')
# Split the RDD 
ratings = data.map(lambda l: l.split('::'))

In [28]:
notSplit = sc.parallelize(data.take(5))
print(test.collect())
# data.collect()
splitedData = sc.parallelize(ratings.take(5))
print(splitedData.collect())
#ratings.collect()

['0::2::3::1424380312', '0::3::1::1424380312', '0::5::2::1424380312', '0::9::4::1424380312', '0::11::1::1424380312']
[['0', '2', '3', '1424380312'], ['0', '3', '1', '1424380312'], ['0', '5', '2', '1424380312'], ['0', '9', '4', '1424380312'], ['0', '11', '1', '1424380312']]


In [29]:
# Transform the ratings RDD 
ratings_final = ratings.map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])))

# Split the data into training and test
training_data, test_data = ratings_final.randomSplit([0.8, 0.2], seed=42)


PythonRDD[168] at RDD at PythonRDD.scala:53


In [31]:
print(sc.parallelize(ratings_final.take(5)))

ParallelCollectionRDD[170] at readRDDFromFile at PythonRDD.scala:274


In [15]:
# Create the ALS model on the training data
model = ALS.train(training_data, rank=5, iterations=5)

# Drop the ratings column 
testdata_no_rating = test_data.map(lambda p: (p[0], p[1]))

# Predict the model  
predictions = model.predictAll(testdata_no_rating)

In [19]:
# Return the first 5 rows of the RDD
predictions.take(10)

[Rating(user=22, product=96, rating=3.3675949167618118),
 Rating(user=24, product=96, rating=2.3279102063647477),
 Rating(user=12, product=96, rating=-0.7927480287646915),
 Rating(user=26, product=96, rating=2.2310481337513237),
 Rating(user=26, product=52, rating=4.860567201568539),
 Rating(user=10, product=52, rating=2.018502533739543),
 Rating(user=2, product=52, rating=1.7615514998127733),
 Rating(user=13, product=52, rating=3.064337101004983),
 Rating(user=19, product=34, rating=0.33194317432083764),
 Rating(user=26, product=4, rating=1.421277716074823)]

In [18]:
# Prepare ratings data
rates = ratings_final.map(lambda r: ((r[0], r[1]), r[2]))

# Prepare predictions data
preds = predictions.map(lambda r: ((r[0], r[1]), r[2]))

# Join the ratings data with predictions data
rates_and_preds = rates.join(preds)

# Calculate and print MSE
MSE = rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error of the model for the test data = {:.2f}".format(MSE))

Mean Squared Error of the model for the test data = 1.82
